In [39]:
import pandas as pd
import csv
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math
import seaborn as sns

In [22]:
income_df = pd.read_csv("2021Census_G02_VIC_SA2.csv")


In [23]:
#selecting necessary columns

income_df = income_df[['SA2_CODE_2021','Median_tot_prsnl_inc_weekly','Median_rent_weekly', 'Median_tot_fam_inc_weekly', 'Median_tot_hhd_inc_weekly' ]]

In [24]:
pop_dfA = pd.read_csv('2021Census_G17A_VIC_SA2.csv')
pop_dfB = pd.read_csv("2021Census_G17B_VIC_SA2.csv")
#pop_dfC = pd.read_csv("2021Census_G17C_VIC_SA2.csv")

In [25]:
pop_dfA = pop_dfA[['SA2_CODE_2021','M_Tot_Tot', 'M_PI_NS_ns_Tot']]

In [26]:
pop_dfB = pop_dfB[['SA2_CODE_2021','F_Tot_Tot', 'F_PI_NS_ns_Tot']]

In [27]:
#pop_dfC = pop_dfC[['SA2_CODE_2021','P_Tot_Tot']]

In [28]:
from functools import reduce
dfs = [pop_dfA, pop_dfB]
population_df = reduce(lambda left,right: pd.merge(left,right,on='SA2_CODE_2021'), dfs)

In [29]:
population_df = population_df.rename({'M_Tot_Tot': 'Male Total', 'M_PI_NS_ns_Tot': 'Male NotStated Total' ,
                           'F_Tot_Tot': 'Female Total', 'F_PI_NS_ns_Tot': 'Female NotStated Total', 
                           }, axis=1)

population_df['total_population'] = population_df['Male Total']+population_df['Male NotStated Total']+population_df['Female Total']+population_df['Female NotStated Total']

In [30]:

population_df=population_df.drop(columns=['Male Total','Male NotStated Total' ,'Female Total', 'Female NotStated Total'])
population_df

,SA2_CODE_2021,total_population
0,201011001,13320
1,201011002,11092
2,201011005,6041
3,201011006,8898
4,201011007,3457
...,...,...
519,217041478,8883
520,217041479,19400
521,217041480,12151
522,297979799,69


In [31]:
df = pd.merge(income_df, population_df, on='SA2_CODE_2021', how="inner")

In [48]:
df[[ 'Median_tot_prsnl_inc_weekly','Median_rent_weekly','Median_tot_fam_inc_weekly','Median_tot_hhd_inc_weekly',
    'total_population']].describe().apply(lambda s: s.apply('{0:.5f}'.format))

,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population
count,524.00000,524.00000,524.00000,524.00000,524.00000
mean,993.86450,357.07824,2143.92939,1760.53626,10866.83397
std,4011.17449,92.54591,632.75876,529.86991,4986.36391
min,0.00000,0.00000,0.00000,0.00000,0.00000
25%,683.75000,310.00000,1742.50000,1401.00000,7259.50000
50%,793.00000,373.00000,2120.50000,1802.50000,10649.50000
75%,918.25000,410.00000,2455.00000,2102.00000,14173.25000
max,92500.00000,612.00000,7000.00000,7000.00000,24673.00000


In [33]:
df_zero_pop = df[df['total_population'] == 0] 

In [34]:
df_zero_pop

,SA2_CODE_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population
85,205021080,0,0,0,0,0
88,205021083,0,0,0,0,0
134,206041127,0,0,0,0,0
139,206041507,92500,0,7000,7000,0


In [35]:
df['Median_tot_prsnl_inc_weekly'].nsmallest(n=10)

85       0
88       0
134      0
214      0
523      0
284    437
287    442
93     444
297    446
398    482
Name: Median_tot_prsnl_inc_weekly, dtype: int64

In [50]:
df_zero_income = df[(df['Median_tot_prsnl_inc_weekly'] == 0) | (df['Median_rent_weekly'] == 0)|
        (df['Median_tot_fam_inc_weekly'] == 0)|
        (df['Median_tot_hhd_inc_weekly'] == 0)]

df_zero_income

,SA2_CODE_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population
85,205021080,0,0,0,0,0
88,205021083,0,0,0,0,0
134,206041127,0,0,0,0,0
139,206041507,92500,0,7000,7000,0
214,208031192,0,486,0,0,22
261,210011227,1125,0,0,0,14
522,297979799,2200,0,0,0,69
523,299999499,0,0,0,0,11082


In [19]:

df.to_csv('income_and_pop.csv', index=False)